# Project 

In [25]:
import games4e
from games4e import Game

In [26]:
import random
random.seed(20)

In [27]:
class Mancala(Game):
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        self.utility = [0] * ((pits_per_player+1) * 2) 
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0
    

    def display(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        
        # write your code here
        if self.current_player == 1:
            if ((self.p1_pits_index[0] <= pit-1) and (pit-1 <= self.p1_pits_index[1]) and self.board[pit-1] > 0):
                self.moves.append((self.current_player, pit))
                return True
            else:
                return False
        else: 
            if ((self.p2_pits_index[0] <= pit+self.pits_per_player-1) and 
                (pit+self.pits_per_player-1 <= self.p2_pits_index[1]) and self.board[pit+self.pits_per_player-1] > 0):
                self.moves.append((self.current_player, pit))
                return True
            else:
                return False
    
    def utility(self, player):
        #Utility=# stones in Max Mancala - # stones in Min Mancala
        # implement the actual utility data struct
        if player == 1:
            return self.board[self.p1_mancala_index] - self.board[self.p2_mancala_index]
        else:
            return self.board[self.p2_mancala_index] - self.board[self.p1_mancala_index]

    

    def actions(self):
        valid_pits = [i for i in range(len(self.board)) if self.valid_move(i)]
        return valid_pits if valid_pits else None
        
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        
        # write your code here
        if self.current_player == 1:
            pits = range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)
        else:
            pits = range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)

        valid_pits = [i + 1 if self.current_player == 1 else i - self.pits_per_player + 1
                  for i in pits if self.board[i] > 0]

        return random.choice(valid_pits) if valid_pits else None
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        # write your code here
        player = self.current_player
        if(pit > self.pits_per_player):
            pit = pit - self.pits_per_player
        #print(f"Player {player} selected pit {pit}")
        if not self.valid_move(pit):
            #print("INVALID MOVE")
            # self.current_player = 3 - self.current_player
            return self.board
        
        if not self.is_terminal():
            if self.current_player == 2:
                pit += self.pits_per_player
            else:  
                pit -= 1
            stones = self.board[pit]
            self.board[pit] = 0
            index = pit

            while stones > 0:
                index = (index + 1) % len(self.board)
                if ((self.current_player == 1 and index == self.p2_mancala_index) or 
                   (self.current_player == 2 and index == self.p1_mancala_index)):
                    continue
                else:
                    self.board[index] += 1
                    stones -= 1

            if ((self.current_player == 1 and self.p1_pits_index[0] <= index <= self.p1_pits_index[1] and self.board[index] == 1) or
           (self.current_player == 2 and self.p2_pits_index[0] <= index <= self.p2_pits_index[1] and self.board[index] == 1)):
                opposite_index = len(self.board) - 2 - index
                if self.board[opposite_index] >= 0:
                    self.board[self.p1_mancala_index if self.current_player == 1 else self.p2_mancala_index] += self.board[opposite_index] + 1
                    self.board[opposite_index] = 0
                    self.board[index] = 0
            self.current_player = 3 - self.current_player
        else:
            print("GAME OVER")
            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                print("Player 1 Wins!")
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                print("Player 2 Wins!")
            else:
                print("It's a Tie!")
        
        return self.board
    
    def is_terminal(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        # write your code here
        p1_empty = all(self.board[i] == 0 for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1))
        p2_empty = all(self.board[i] == 0 for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1))

        if p1_empty or p2_empty:
            if not p1_empty:
                for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                    self.board[self.p1_mancala_index] += self.board[i]
                    self.board[i] = 0
                    
            if not p2_empty:
                for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                    self.board[self.p2_mancala_index] += self.board[i]
                    self.board[i] = 0
                    
            return True
        
        return False


In [28]:
import random

p1_wins = 0
p2_wins = 0
ties = 0
total_turns = 0

for game_num in range(100):
    
    game = Mancala()
    turns = 0
    amount_of_moves = 0

    while not game.is_terminal():
        move = game.random_move_generator()

        
        if move is None:
            game.is_terminal()
            break

        game.play(move)
        turns += 1

        amount_of_moves += 1
        
        if amount_of_moves > 300:
            break

    total_turns += turns

    p1_score = game.board[game.p1_mancala_index]
    p2_score = game.board[game.p2_mancala_index]

    if p1_score > p2_score:
        p1_wins += 1
        
    elif p2_score > p1_score:
        p2_wins += 1
        
    else:
        ties += 1
        

print(f"Player 1 Wins: {p1_wins} ({p1_wins}%)")
print(f"Player 2 Wins: {p2_wins} ({p2_wins}%)")
print(f"Ties: {ties} ({ties}%)")
print(f"Average number of turns per game: {total_turns / 100:.2f}")

Player 1 Wins: 50 (50%)
Player 2 Wins: 47 (47%)
Ties: 3 (3%)
Average number of turns per game: 45.28
